<a href="https://colab.research.google.com/github/hdavidoff/datathon23/blob/hannah/from_new_art_prices_to_tweakable_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
artworks = pd.read_parquet('https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/data/Artwork.parquet.gzip')
artists = pd.read_parquet('https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/data/Artist.parquet.gzip')

import io
import matplotlib.pyplot as plt
import requests
from PIL import Image

def display_artwork(url):
    response = requests.get(url)
    image = Image.open(io.BytesIO(response.content))
    plt.imshow(image)
    plt.axis("off")
    plt.show()

def get_AI_gen(url):
    response = requests.get(url)
    image = Image.open(io.BytesIO(response.content))
    return image

In [1]:
!pip install git+https://github.com/LIAAD/yake
import yake
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')   # Required for tokenization
nltk.download('wordnet') # Required for lemmatization
nltk.download('omw-1.4')


import numpy as np
from wordcloud import WordCloud
from collections import Counter
# Instantiate
lemmatizer = WordNetLemmatizer()


## code source: https://medium.com/@y.s.yoon/nlp-illustration-in-python-extracting-keywords-e9c4a6e0a267

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/LIAAD/yake to /tmp/pip-req-build-drg5dl33
  Running command git clone --filter=blob:none --quiet https://github.com/LIAAD/yake /tmp/pip-req-build-drg5dl33
  Resolved https://github.com/LIAAD/yake to commit 8d71d94ded93fb77f1361f62e5264f19b9c91cd7
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 KB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for yake: filename=yake-0.4.8-py2.py3-none-any.whl size=62600 sha256=ae4af93f8fb76e68a0a4c3bb3e188232ed6c91ad6ee4484bfe11c50148cd8ec0
  Stored in directory: /tmp/pip-ephem-wheel-cache-8f5ejtz1/wheels/91/6a/18/ab7d2fc1e58b6ed936dbe90881425aea909bae3f98dae32f66
  Created wheel for jellyfish: filename=jellyfish-0.9.0-cp38-cp38-linux_x86_64.whl size=77926 sha256=287db5c116692160a56276a8ea053764d815b826689a837cff8648088707f961
  Stored in director

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [14]:
## first pre-process
## lowercase
## remove punctuation and number
## tokenize
## remove stop words
## lemmatization
nltk.download('stopwords')
# Exclusion list of punctuations and numbers
exclist = string.punctuation + string.digits + '-'+'’'+'”'+'–'
# Print the exclusion list
print(exclist)


!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~0123456789-’”–


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [15]:
## count number of matching words between name and price name
def count_match_words(text1,text2):
  tokens1 = word_tokenize(text1)
  tokens2 = word_tokenize(text2)
  count_match = [1  for token in tokens1 if token in tokens2]
  sum_match = sum(count_match)
  perc_match = (sum_match/len(tokens1))*100

  return perc_match

# NEW SOURCE PRICES

Upload the .csv to wherever you're running this

In [13]:
art_with_prices = pd.read_csv('/content/art_with_prices22feb.csv')
## first get all with non-null prices and summaries
art_with_ps = art_with_prices.loc[(art_with_prices['price_name'].notnull())&(art_with_prices['summary'].notnull())]
art_with_ps.count()

Unnamed: 0      907
id              907
name            907
url             907
image_url       907
artist          907
rating          447
summary         907
year             73
medium          132
location        149
name_english    907
artist_name     907
price           907
price_name      907
dtype: int64

In [10]:
art_filt_ln2 = art_with_ps
art_filt_ln2['ln_split'] = art_filt_ln2['artist_name'].str.split(' ')
art_filt_ln2['last_name'] = art_filt_ln2.ln_split.apply(lambda x:x[-1])
art_filt_2_ln = art_filt_ln2
art_filt_2_ln = art_filt_2_ln[~art_filt_2_ln['summary'].isnull()]
art_filt_2_ln['ln_in_sum'] = art_filt_2_ln.apply(lambda x: x.last_name in x.summary, axis =1)
for_summ = art_filt_2_ln.loc[art_filt_2_ln['ln_in_sum']==True].copy() ## take rows where last name of artist is present in summary
for_summ = for_summ.drop_duplicates(subset='summary') # keeps first ## don't analyze the art from this limited set as it could throw out paintings we could otherwise keep
for_summ.count()

NameError: ignored

Clean the text from the name and price_name columns for matching.

In [ ]:
from nltk.corpus import stopwords
stop_words = stopwords.words("english")
stop_words.append('du')
stop_words.append('au')
stop_words.append('de')
stop_words.append('à')
stop_words.append('la')

# Exclusion list of punctuations and numbers
exclist = string.punctuation + string.digits + '-'+'’'+'”'+'–'
# Print the exclusion list
print(exclist)


def clean_text(text):
  text = text.lower()

  text = text.translate(str.maketrans("","",exclist))

 
  # Tokenization
  tokens = word_tokenize(text)

  # Lemmatization
  tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
  # Remove stop words
  tokens = [token for token in tokens if token not in stop_words]

  #special replace
  st_replace = ['saint']
  tokens = [token if token not in st_replace else 'st' for token in tokens ]
    
  # Join tokens
  clean_text = " ".join(tokens)
    
  # Return the output
  return clean_text

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~0123456789-’”–


In [ ]:
for_summ['name_english'] = for_summ.apply(lambda row: clean_text(row['name_english']),axis=1)
for_summ['price_name'] = for_summ.apply(lambda row: clean_text(row['price_name']),axis=1)
for_summ.head()

,id,name,url,image_url,artist,rating,summary,year,medium,location,name_english,artist_name,price,price_name,ln_split,last_name,ln_in_sum
5,5,Blossoming Almond Tree,http://wikigallery.org/wiki/painting_14446/Vin...,https://kuleuven-datathon-2023.s3.eu-central-1...,0,10.0,Almond Blossoms is a group of several painting...,1890.0,0.0,Van Gogh Museum,blossoming almond tree,Vincent Van Gogh,50.0,almond blossom glass,"[Vincent, Van, Gogh]",Gogh,True
8,8,Agostina Segatori Sitting In The Cafe Du Tambo...,http://wikigallery.org/wiki/painting_14352/Vin...,https://kuleuven-datathon-2023.s3.eu-central-1...,0,9.0,Agostina Segatori Sitting in the Café du Tambo...,1887.0,0.0,Van Gogh Museum,agostina segatori sitting cafe tambourin,Vincent Van Gogh,669500.0,peasant woman halffigure sitting,"[Vincent, Van, Gogh]",Gogh,True
10,10,Peach Trees in Blossom,http://wikigallery.org/wiki/painting_133052/Vi...,https://kuleuven-datathon-2023.s3.eu-central-1...,0,NaN,Peach Trees in Blossom is an 1899 painting by ...,NaN,NaN,NaN,peach tree blossom,Vincent Van Gogh,110500.0,olive tree yellow sky vincent van gogh picture...,"[Vincent, Van, Gogh]",Gogh,True
16,16,Crab On Its Back,http://wikigallery.org/wiki/painting_14523/Vin...,https://kuleuven-datathon-2023.s3.eu-central-1...,0,NaN,Crab on its Back (Dutch: Een op zijn rug ligge...,NaN,0.0,Van Gogh Museum,crab back,Vincent Van Gogh,692248.0,house seen back window siens mother house,"[Vincent, Van, Gogh]",Gogh,True
18,18,Fishing Boats On The Beach At Saintes Maries,http://wikigallery.org/wiki/painting_14474/Vin...,https://kuleuven-datathon-2023.s3.eu-central-1...,0,9.0,Saintes-Maries is the subject of a series of p...,NaN,NaN,NaN,fishing boat beach saintes mary,Vincent Van Gogh,5062500.0,souvenir saintesmaries mediterrane boat beach ...,"[Vincent, Van, Gogh]",Gogh,True


In [ ]:
for_summ['match_perc'] = for_summ.apply(lambda row: count_match_words(row['name_english'],row['price_name']),axis=1)
for_summ.head()

,id,name,url,image_url,artist,rating,summary,year,medium,location,name_english,artist_name,price,price_name,ln_split,last_name,ln_in_sum,match_perc
5,5,Blossoming Almond Tree,http://wikigallery.org/wiki/painting_14446/Vin...,https://kuleuven-datathon-2023.s3.eu-central-1...,0,10.0,Almond Blossoms is a group of several painting...,1890.0,0.0,Van Gogh Museum,blossoming almond tree,Vincent Van Gogh,50.0,almond blossom glass,"[Vincent, Van, Gogh]",Gogh,True,33.333333
8,8,Agostina Segatori Sitting In The Cafe Du Tambo...,http://wikigallery.org/wiki/painting_14352/Vin...,https://kuleuven-datathon-2023.s3.eu-central-1...,0,9.0,Agostina Segatori Sitting in the Café du Tambo...,1887.0,0.0,Van Gogh Museum,agostina segatori sitting cafe tambourin,Vincent Van Gogh,669500.0,peasant woman halffigure sitting,"[Vincent, Van, Gogh]",Gogh,True,20.000000
10,10,Peach Trees in Blossom,http://wikigallery.org/wiki/painting_133052/Vi...,https://kuleuven-datathon-2023.s3.eu-central-1...,0,NaN,Peach Trees in Blossom is an 1899 painting by ...,NaN,NaN,NaN,peach tree blossom,Vincent Van Gogh,110500.0,olive tree yellow sky vincent van gogh picture...,"[Vincent, Van, Gogh]",Gogh,True,33.333333
16,16,Crab On Its Back,http://wikigallery.org/wiki/painting_14523/Vin...,https://kuleuven-datathon-2023.s3.eu-central-1...,0,NaN,Crab on its Back (Dutch: Een op zijn rug ligge...,NaN,0.0,Van Gogh Museum,crab back,Vincent Van Gogh,692248.0,house seen back window siens mother house,"[Vincent, Van, Gogh]",Gogh,True,50.000000
18,18,Fishing Boats On The Beach At Saintes Maries,http://wikigallery.org/wiki/painting_14474/Vin...,https://kuleuven-datathon-2023.s3.eu-central-1...,0,9.0,Saintes-Maries is the subject of a series of p...,NaN,NaN,NaN,fishing boat beach saintes mary,Vincent Van Gogh,5062500.0,souvenir saintesmaries mediterrane boat beach ...,"[Vincent, Van, Gogh]",Gogh,True,40.000000


In [16]:
import seaborn as sns
sns.histplot(x=for_summ['match_perc'])
plt.xticks(rotation='vertical')
plt.title("% title match", fontsize = 20)
plt.show()

NameError: ignored

In [ ]:
#set threshold for percentage max minimum
min_match = 40
clean_for_summ = for_summ.loc[for_summ['match_perc']>=min_match]
clean_for_summ.count()

id              229
name            229
url             229
image_url       229
artist          229
rating          128
summary         229
year             61
medium          100
location        103
name_english    229
artist_name     229
price           229
price_name      229
ln_split        229
last_name       229
ln_in_sum       229
match_perc      229
dtype: int64